In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

# Load your training dataset
train_data = pd.read_csv("train.csv")  # Replace with your training dataset path

# Initialize a dictionary to store models, reports, and accuracies for each vehicle
vehicle_models = {}
vehicle_reports = {}
vehicle_accuracies = {}

# Convert categorical target labels to numeric values
label_encoder = LabelEncoder()
train_data['Alert'] = label_encoder.fit_transform(train_data['Alert'])

# Define a deep learning model using TensorFlow/Keras
def create_model(input_shape, num_classes):
    model = keras.Sequential([
        layers.Input(shape=input_shape),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Loop through each unique vehicle ID
unique_vehicles = train_data['Vehicle'].unique()
for vehicle_id in unique_vehicles:
    # Filter training data for the current vehicle
    vehicle_data = train_data[train_data['Vehicle'] == vehicle_id]

    # Separate features and target
    X = vehicle_data[['Lat', 'Long', 'Vehicle', 'Speed']]
    y = vehicle_data['Alert']

    # Split data into train and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create a KerasClassifier
    input_shape = (X_train.shape[1],)
    num_classes = len(label_encoder.classes_)
    model = create_model(input_shape, num_classes)

    # Define hyperparameters
    epochs = 100
    batch_size = 64

    # Train the model
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, validation_data=(X_val, y_val))

    # Evaluate the model on the validation set
    val_loss, val_accuracy = model.evaluate(X_val, y_val)
    print(f"Validation Accuracy for Vehicle {vehicle_id}: {val_accuracy:.4f}")

    # Store the trained model for the current vehicle
    vehicle_models[vehicle_id] = model

# Load your testing dataset
test_data = pd.read_csv("test.csv")  # Replace with your testing dataset path

# Initialize a list to store predicted alerts
predicted_alerts = []

# Real-time monitoring and feedback loop
for index, row in test_data.iterrows():
    vehicle_id = row['Vehicle']
    model = vehicle_models.get(vehicle_id)
    if model:
        # Prepare input features for prediction
        features = np.array([[row['Lat'], row['Long'], row['Vehicle'], row['Speed']]])

        # Make the prediction
        predicted_alert = model.predict(features)

        # Convert predicted class indices back to categorical labels
        predicted_alert_cleaned = label_encoder.inverse_transform(np.argmax(predicted_alert, axis=1))

        # Append the predicted alert to the list
        predicted_alerts.append(predicted_alert_cleaned[0])

        # Check if the predicted alert matches a predefined behavior pattern
        behavior_patterns = {
            'cas_hmw': "You are frequently exceeding the speed limit. Please reduce your speed for safety.",
            'cas_ldw': "Your lane discipline needs improvement. Use turn signals and stay in your lane.",
            'cas_fcw': "You are following too closely to the vehicle ahead. Maintain a safe following distance.",
            'cas_pcw': "Your following distance is too short. Keep a safe distance from the vehicle in front."
        }
        if predicted_alert_cleaned[0] in behavior_patterns:
            # Provide real-time feedback to the driver based on behavior pattern
            coaching_message = behavior_patterns[predicted_alert_cleaned[0]]
            print(f"Driver ID {vehicle_id}: {coaching_message}")

# Save the updated test dataset to a new CSV file
test_data['PredictedAlert'] = predicted_alerts
test_data.to_csv("test_with_predicted_alerts.csv", index=False)


42/42 [==============================] - 0s 2ms/step - loss: 0.9090 - accuracy: 0.6075
Validation Accuracy for Vehicle 805: 0.6075
35/35 [==============================] - 0s 2ms/step - loss: 1.0006 - accuracy: 0.4717
Validation Accuracy for Vehicle 2846: 0.4717
18/18 [==============================] - 0s 2ms/step - loss: 1.0765 - accuracy: 0.4973
Validation Accuracy for Vehicle 3143: 0.4973
36/36 [==============================] - 0s 987us/step - loss: 0.8703 - accuracy: 0.7110
Validation Accuracy for Vehicle 5339: 0.7110
1/1 [==============================] - 0s 111ms/step
Driver ID 805: You are frequently exceeding the speed limit. Please reduce your speed for safety.
1/1 [==============================] - 0s 14ms/step
Driver ID 805: You are frequently exceeding the speed limit. Please reduce your speed for safety.
1/1 [==============================] - 0s 13ms/step
Driver ID 805: You are frequently exceeding the speed limit. Please reduce your speed for safety.
1/1 [===============